<h1 style="color:#189AB4"> Initiation au Machine Learning - Visualisation et nettoyage de données</h2>

<h2>But de l'exercice</h2>

<p>Afin d'éviter qu'un prêt ne devienne un défaut de paiement, les banques doivent trouver comment
les prédire en exploitant les données basées sur les comportements des clients. Les modèles d'apprentissage automatique semblent être l'une des solutions les plus efficaces pour prévoir les défauts de paiement. Par conséquent, l'objectif de ce projet est de construire des modèles supervisés pour les prédictions de défaut de prêt et d'explorer les impacts des facteurs comportementaux des clients sur la poursuite des prévisions.</p>

<h2>Livrables</h2>

- Un notebook : visualisation et de nettoyage de donnéesUn
- notebook : modélisation

<h2>Importation des bibliothèques</h2>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<h2>Lecture du fichier csv</h2>

In [ ]:
train = pd.read_csv('train.csv')

<h2>Création d'une copie du fichier csv</h2>

In [ ]:
train_copie = train.copy()

<i style="color:#90ADC6">-> je crée des copy des fichiers csv originaux car je ne veux pas écraser l'original.</p>

<h2>Affichage et vérifications des données</h2>

In [ ]:
pd.set_option('display.max_columns', None) #affiche toutes les colonnes
train_copie.head()

In [ ]:
train_copie.shape

In [ ]:
train_copie.info()

In [ ]:
train_copie.describe()

In [ ]:
train_copie.duplicated().sum()

<i style="color:#90ADC6">->Il y a 3 doublons dans ce dataset, les lignes sont identiques, je penses que ce n'est pas pertinent de garder ces données, mais je ne les supprime pas pour le moment, il faut que je fasse plus de recherche pour prendre une décision final.</i>

In [ ]:
train_copie.dtypes

<i style="color:#90ADC6">->Au total, il y a 2100 transactions. Parmis ces données, 612 lignes ne sont pas des prêts je pense où alors les données de prêt sont manquantes</i>

<h2>Vérification des valeurs nulles</h2>

In [ ]:
train_copie.isnull().sum()

<i style="color:#90ADC6">->Je vois qu'il y a des lignes en trop dans DueDate, LoanApplicationId, ThirdPartyId, et je trouve ça anormal, par exemple, pourquoi est-ce qu'il y a 614 dates limite de remboursement (DueDate) mais seulement 612 id de transaction de prêt (LoanId), il y a des lignes de trop. </p>

In [ ]:
train_null = train_copie[train_copie.isna().any(axis=1)]
train_null

<i style="color:#90ADC6">->J'affiche les lignes qui ont au moins 1 "nan". Je vois que tous les gens qui ont un "nan" ont transactionstatus à 0, donc je vais explorer cette colonne.</p>

In [ ]:
train_null['TransactionStatus'].value_counts()

<i style="color:#90ADC6">->Je vois qu'il y a 9 valeurs à 1, donc 9 lignes avec un prêt accepté, mais qui ont des valeurs manquantes dans certaines colonnes, ce qui est anormal. Je vais afficher ces 9 lignes suspects.</p>

In [ ]:
train_null.loc[train_null['TransactionStatus'] == 1]

<i style="color:#90ADC6">->Parmis les 9 lignes affichés, je vois le CustomerId_258 avec 3 transactions sans LoandApplicationId mais qui ont le même BatchId, donc c'est peut-être un paiement en plusieurs fois, un remboursement. Je pense que ce n'est pas une ligne de demande de prêt, la demande de prêt doit être sur une autre ligne à mon avis. Ce n'est qu'une hypothèse. </p>

In [ ]:
train_copie[train_copie['CustomerId'] == "CustomerId_258"]

In [ ]:
train_null.loc[train_null['TransactionId'] == "TransactionId_1041"]

In [ ]:
train_null.loc[train_null['BatchId'] == "BatchId_1970"]

<i style="color:#90ADC6">->Ma théorie qui disait que la demande de prêt doit être sur une autre ligne est fausse. J'affiche toutes les transactions de la personnes qui ont le même BatchId ou TransactionId mais il y en a pas. Ou alors je recherche la mauvaise donnée.</p>

In [ ]:
train_copie[(train_copie['CustomerId'] == 'CustomerId_258') & (train_copie['LoanApplicationId'].notna())]

<i style="color:#90ADC6">->J'affiche les transactions du CustomerId_258 qui n'ont pas de LoanApplicationId, donc pas de demande de prêt.</p>

<h2>Modifications des types de données incorrectes</h2>

In [ ]:
train_copie[['TransactionStartTime', 'IssuedDateLoan', 'PaidOnDate', 'DueDate']] = train_copie[['TransactionStartTime', 'IssuedDateLoan', 'PaidOnDate', 'DueDate']].apply(pd.to_datetime, errors='coerce')
train_copie[['TransactionStatus', 'IsFinalPayBack', 'IsThirdPartyConfirmed', 'IsDefaulted']] = train_copie[['TransactionStatus', 'IsFinalPayBack', 'IsThirdPartyConfirmed', 'IsDefaulted']].fillna(0).astype(int)

<i style="color:#90ADC6">-> Après une analyse des types de données, je vois que certaines colonnes n'ont pas le bon type de données, donc je les convertis. J'ai aussi dû les convertir rapidement car j'avais besoin d'effectuer des calcules mais les colonnes n'avaient pas le bon type, donc ça me mettait une erreur.</p>

<h2>Affichage des clients avec un retard de paiement</h2>

In [ ]:
train_copie['days_late'] = (train_copie['PaidOnDate'] - train_copie['DueDate']).dt.days
train_copie

<i style="color:#90ADC6">->Ici, je crée une nouvelle colonne qui affiche le nombre de jour en retard ou non d'un remboursement de prêt. </p>

In [259]:
retard_paiement = train_copie[train_copie['days_late'] > 0]
retard_paiement

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,TransactionStatus,IssuedDateLoan,AmountLoan,Currency,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted,days_late
26,CustomerId_474,2018-10-26 09:19:00,10000.0,-10000.0,TransactionId_2299,BatchId_500,SubscriptionId_1,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2018-10-26 09:19:00,10000.0,UGX,LoanId_934,2019-05-16 07:25:00,1,InvestorId_2,2018-11-25 09:18:00,LoanApplicationId_164,PayBackId_1899,ThirdPartyId_601,0,1,171.0
28,CustomerId_432,2018-10-26 16:12:00,31766.0,-30840.0,TransactionId_2128,BatchId_381,SubscriptionId_1,UGX,256,ProviderId_1,ProductId_6,tv,ChannelId_1,1,2018-10-26 16:12:00,30840.0,UGX,LoanId_386,2019-04-26 16:24:00,1,InvestorId_2,2018-11-25 16:12:00,LoanApplicationId_836,PayBackId_1262,ThirdPartyId_1546,1,0,152.0
35,CustomerId_405,2018-11-01 16:12:00,53500.0,-50000.0,TransactionId_549,BatchId_2426,SubscriptionId_1,UGX,256,ProviderId_1,ProductId_15,utility_bill,ChannelId_1,1,2018-11-01 16:12:00,63398.0,UGX,LoanId_77,2018-12-06 09:26:00,0,InvestorId_2,2018-12-01 16:12:00,LoanApplicationId_534,PayBackId_165,ThirdPartyId_1760,1,0,4.0
36,CustomerId_405,2018-11-01 16:12:00,53500.0,-50000.0,TransactionId_549,BatchId_2426,SubscriptionId_1,UGX,256,ProviderId_1,ProductId_15,utility_bill,ChannelId_1,1,2018-11-01 16:12:00,2942.0,UGX,LoanId_77,2018-12-11 17:00:00,1,InvestorId_2,2018-12-01 16:12:00,LoanApplicationId_534,PayBackId_1908,ThirdPartyId_1716,1,0,10.0
37,CustomerId_474,2018-11-01 17:56:00,5000.0,-5000.0,TransactionId_1452,BatchId_728,SubscriptionId_1,UGX,256,ProviderId_1,ProductId_3,airtime,ChannelId_1,1,2018-11-01 17:56:00,500.0,UGX,LoanId_1554,2019-07-15 09:12:00,0,InvestorId_2,2018-12-01 17:56:00,LoanApplicationId_937,PayBackId_1295,ThirdPartyId_1702,1,1,225.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079,CustomerId_180,2019-03-29 04:43:00,5000.0,-5000.0,TransactionId_1575,BatchId_359,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_1,airtime,ChannelId_1,1,2019-03-29 04:43:00,6325.0,UGX,LoanId_409,2019-05-13 09:33:00,1,InvestorId_1,2019-04-28 04:43:00,LoanApplicationId_1610,PayBackId_187,ThirdPartyId_1891,1,0,15.0
2081,CustomerId_180,2019-03-29 06:15:00,4000.0,-4000.0,TransactionId_1010,BatchId_614,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_3,airtime,ChannelId_1,1,2019-03-29 06:15:00,4860.0,UGX,LoanId_247,2019-05-03 18:31:00,1,InvestorId_1,2019-04-28 06:15:00,LoanApplicationId_747,PayBackId_246,ThirdPartyId_607,1,0,5.0
2083,CustomerId_258,2019-03-29 08:13:00,10000.0,-10000.0,TransactionId_2464,BatchId_417,SubscriptionId_1,UGX,256,ProviderId_1,ProductId_3,airtime,ChannelId_1,1,2019-03-29 08:13:00,11950.0,UGX,LoanId_127,2019-04-29 09:52:00,1,InvestorId_2,2019-04-28 08:13:00,LoanApplicationId_1035,PayBackId_1044,ThirdPartyId_550,1,0,1.0
2087,CustomerId_502,2019-03-29 14:43:00,2000.0,-2000.0,TransactionId_309,BatchId_777,SubscriptionId_5,UGX,256,ProviderId_1,ProductId_3,airtime,ChannelId_1,1,2019-03-29 14:43:00,2550.0,UGX,LoanId_848,2019-05-16 08:04:00,1,InvestorId_2,2019-04-28 14:43:00,LoanApplicationId_252,PayBackId_151,ThirdPartyId_646,0,0,17.0


<i style="color:#90ADC6">-> Je vois que certaines lignes ont dépasser le délais de paiement mais IsDefaulted est à 0, ce qui est illogique car il est censé être à 1 puisque ils n'ont pas payé à temps.</p>

In [ ]:
#afficher les lignes qui ont un retard de paiement mais qui sont pas en defaut IsDefaulted (0)
train_copie.loc[(train_copie['IsDefaulted'] == 0) & (train_copie['PaidOnDate'] > train_copie['DueDate'])]

<i style="color:#90ADC6"> ->Ici, je m'apperçois qu'il y a 341 anomalies. Ils sont pas en défaut mais ont un retard de paiement.  </p>

In [ ]:
train_copie.loc[(train_copie['IsDefaulted'] == 1) & (train_copie['IsFinalPayBack'] == 0)]

<i style="color:#90ADC6"> -> </p>

In [ ]:
train_nvl_colonne = train_copie.copy()

In [ ]:
train_nvl_colonne['NombreTransactions'] = train_nvl_colonne.groupby('CustomerId')['TransactionId'].transform('count')

In [ ]:
nb_prets = train_nvl_colonne[train_nvl_colonne['AmountLoan'].notnull()].groupby('CustomerId').size().rename('NombrePretsParClient')
train_nvl_colonne = train_nvl_colonne.merge(nb_prets, on='CustomerId', how='left')
train_nvl_colonne['NombrePretsParClient'] = train_nvl_colonne['NombrePretsParClient'].fillna(0).astype(int)

In [ ]:
train_nvl_colonne.head()

In [ ]:
train_copie[train_copie['CustomerId'] == "CustomerId_27"].reset_index(drop=True)

<h2>Vérification des corélations entre les colonnes</h2>

In [ ]:
pearson_corr = train_copie.select_dtypes(include=['int64', 'float']).corr(method='pearson')
pearson_corr

In [ ]:
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm')
plt.title("Pearson Correlation Heatmap")
plt.show()

In [ ]:
train_nvl_colonne.to_csv('train_copie.csv', index=False)